In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
import csv
from keras.models import load_model
model = load_model('temp_RIV2_1.h5')
# img_path="data/val_data/id_1_labels_[62, 17, 66, 214, 105, 137, 85].jpg"

# im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
# imResize = im.resize((299, 299))
# imageData = np.array(imResize)

# X = imageData.reshape(1,299,299,3).astype('float32')

# pred = model.predict(X)
# class_i = np.argmax(pred[0])
# prob="%0.2f"%(pred[0][class_i])

Using TensorFlow backend.


In [14]:
img_path="data/val_data/id_1_labels_[62, 17, 66, 214, 105, 137, 85].jpg"

im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
imResize = im.resize((299, 299))
imageData = np.array(imResize)

X = imageData.reshape(1,299,299,3).astype('float32')

pred = model.predict(X)



In [15]:
pred_arr=np.array(pred)

In [2]:
pred_arr

In [20]:
for i in range(0,228):
    if(pred[0][i]>0.2):
        print(i+1)

17
66
105
153
171
214


In [35]:
for i in range(0,228):
    if(pred[0][0][i]>0.1):
        print(i)

16
18
19
52
61
65
104
105
136
137
152
163
170
213
221


In [11]:
img_name="id_64_labels_[47, 66, 226, 148, 178, 171, 137, 115, 20, 175, 138].jpg"
temp=img_name.split("_")
temp=temp[3].split(".")
temp=temp[0].split("[")
temp=temp[1].split("]")
temp=temp[0].split(",")

each_label_list = [0] * 228
for j in range(0,len(temp)):
    each_label_list[int(temp[j])-1]=1

In [12]:
train_data = pd.read_csv('data/train.csv')   ####1014544
val_data = pd.read_csv('data/val.csv')
#test_data = pd.read_csv('data/test.csv')
file_dir="data/train_data/"
val_dir="data/val_data/"

images = []
labels = []
for img_num in range(0,9897):
    img_name=val_data['filename'][img_num]
    img_path = val_dir+img_name
    if os.path.exists(img_path):
        im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
        imResize = im.resize((299, 299))
        imageData = np.array(imResize)
        images.append(imageData)
        #(filename,extension) = os.path.splitext(img_name)

        #labels.append(train_data['landmark_id'][img_num])
        temp=img_name.split("_")
        temp=temp[3].split(".")
        temp=temp[0].split("[")
        temp=temp[1].split("]")
        temp=temp[0].split(",")

        each_label_list = [0] * 228
        for j in range(0,len(temp)):
            each_label_list[int(temp[j])-1]=1

        labels.append(each_label_list)

X_val = np.array(images)
X_val = X_val.reshape(X_val.shape[0],299,299,3).astype('float32')
y_val=np.array(labels,dtype=np.float)

In [3]:
preds=model.predict(X_val, batch_size=32, verbose=1)

9897/9897 [==============================] - 76s 8ms/step


In [6]:
preds[0][227]

0.007901025

In [1]:
for i in range(0,9897):
    predList=[]
    for j in range(0,228):
        if(preds[i][j]>0.3):
            predList.append(j+1)
    #print(predList)

In [13]:
model.evaluate(X_val, y_val, batch_size=32, verbose=1)

9897/9897 [==============================] - 179s 18ms/step


[0.10756188476430149, 0.96896543084364584]

In [1]:
#####预测并生成submission.csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
import csv
from keras.models import load_model
import json


test_dir="data/test_data/"

model = load_model('temp_RIV2_1.h5')

file_path="data/temp_RIV2_1.csv"

for _id in range(1,39707):
    
    img_path=test_dir+"%d"%(_id)+".jpg"
    im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
    imResize = im.resize((299, 299))
    imageData = np.array(imResize)

    X = imageData.reshape(1,299,299,3).astype('float32')
    
    res=""
    pred = model.predict(X)
    for i in range(0,228):
        if(pred[0][i]>0.2):
            res+="%d "%(i+1)
    
    
    if os.path.exists(file_path):
        da = [_id,res]
        # 写入数据
        csvFile = open(file_path, "a+")
        writer = csv.writer(csvFile)
        # 写入的内容都是以列表的形式传入函数
        writer.writerow(da)
        csvFile.close()
    else:
        fileHeader = ["image_id", "label_id"]
        # 假设我们要写入的是以下两行数据
        da = [_id,res]
        # 写入数据
        csvFile = open(file_path, "a+")
        writer = csv.writer(csvFile)
        # 写入的内容都是以列表的形式传入函数
        writer.writerow(fileHeader)
        writer.writerow(da)
        csvFile.close()

Using TensorFlow backend.
